In [34]:
import random
import csv

# Listes de noms et prénoms
noms = ["Manzombi", "Kukwabantu", "Yoloyolo", "Andedi", "Nzeba", "Mbala", "Maketa", "Kalala", "Nsapu", "Kabong", "Kahindo", "Etumba", "Otshudi", "Yav", "Amani", "Buana", ]
prenoms = ["Rachel", "Joel", "Mariella", "Elvira", "Devernay", "Joseph",  "Alice", "Bob", "Charlie", "Diana", "Eve", "Frank", "Gina", "Hugo", "Iris", "Jack"]

# Liste des cours
cours = ["Math", "Physique", "Programmation parallele", "Finances bubliques", "Projet", "Algo", "c#", "Java EE", "Cloud Computing"  ]

# Fonction pour générer une note aléatoire
def generer_note():
  return random.randint(0, 20)

# Fonction pour générer un nom et un prénom aléatoires
def generer_nom_complet():
  nom = random.choice(noms)
  prenom = random.choice(prenoms)
  return nom + " " + prenom

# Ouvrir un fichier CSV en écriture
with open("etudiants_notes_10000.csv", "w", newline="") as fichier_csv:

  # Ecrire l'en-tête
  ecrivain_csv = csv.writer(fichier_csv)
  ecrivain_csv.writerow(["Nom complet"] + cours)

  # Pour 10000 étudiants
  for _ in range(10):

    # Générer le nom complet
    nom_complet = generer_nom_complet()

    # Générer les notes
    notes = [generer_note() for _ in range(len(cours))]

    # Ecrire la ligne dans le fichier CSV
    ecrivain_csv.writerow([nom_complet] + notes)


Calculons le pourcentage de chaque candidat séquentiellement

In [37]:
import csv

# Fonction pour calculer le pourcentage d'un étudiant
def calculer_pourcentage(notes):
  total = sum(notes)
  pourcentage = (total / (len(notes) * 20)) * 100
  # print(str(total) + "/" + str(len(notes) * 20))
  # print(pourcentage)
  return round(pourcentage, 2)


# Ouvrir le fichier CSV en lecture
with open("etudiants_notes_10000.csv", "r", newline="") as fichier_csv:

  # Lire le fichier ligne par ligne
  lecteur_csv = csv.reader(fichier_csv)

  # Sauter l'en-tête
  next(lecteur_csv, None)

  # Pour chaque ligne
  for ligne in lecteur_csv:

    # Extraire les notes
    notes = [int(note) for note in ligne[1:]]

    # Calculer le pourcentage
    pourcentage = calculer_pourcentage(notes)

    # Afficher le nom de l'étudiant et son pourcentage
    print(f"{ligne[0]}: {pourcentage}%")


Nsapu Charlie: 55.56%
Kukwabantu Bob: 41.11%
Yav Joseph: 47.22%
Amani Rachel: 39.44%
Kukwabantu Alice: 48.89%
Buana Joel: 51.67%
Nzeba Mariella: 43.89%
Etumba Hugo: 65.56%
Mbala Diana: 59.44%
Manzombi Gina: 32.22%


Utlilisons des processus pour pour paralleliser le calcul en attribuant à chaque processus un lot particulier de données en fonctions du nombre de données à traiter et du nombre de processus. Par exemple pour trois processus devant traiter 10 données nous
``` 
 taille_lot = données à traiter / nombre de processus 
            = 10 / 3 
            = 3
```
Pour chaque processus il y'aura environ 3 donnée à traiter.  Ainsi nous auront des intervals avec des début et des fins espacés de 3 pour la plupart

- debuts = [0, 3, 6]
- fins = [2, 5, 10]

Chaque processus executera de ce fait la fonction de traitement avec ses paramètres début et fun

- process_1 -> 0, 2
- process_2 -> 3, 5
- process_1 -> 6, 10


In [1]:
import csv
import multiprocessing
from itertools import islice 

# Fonction pour calculer le pourcentage d'un étudiant
def calculer_pourcentage(notes):
  total = sum(notes)
  pourcentage = (total / (len(notes) * 20)) * 100
  return round(pourcentage, 2)

# Fonction pour traiter un lot d'étudiants
def traiter_lot(fichier_csv, noms, debut, fin):
  with open(fichier_csv, "r", newline="") as fichier:
    lecteur_csv = csv.reader(fichier)
    # Sauter l'en-tête
    lecteur_csv = islice(lecteur_csv, 1, None)
    # Itérer sur les étudiants du lot
   
    for i, ligne in enumerate(lecteur_csv):
      # print(debut)
      # print(fin)
      # print("\n")
      if i not in range(debut, fin + 1 ): 
        continue
      # Extraire les notes
      notes = [int(note) for note in ligne[1:]]
      # Calculer le pourcentage
      pourcentage = calculer_pourcentage(notes)
      # Afficher le nom de l'étudiant et son pourcentage
      print(f"{noms[i]}: {pourcentage} % \n")

# Fonction principale
def main():
  # Ouvrir le fichier CSV des étudiants en lecture
  with open("etudiants_notes_10000.csv", "r", newline="") as fichier_csv:
    # Lire les noms des étudiants
    lecteur_csv = islice(csv.reader(fichier_csv), 1, None)

    noms = [ligne[0] for ligne in lecteur_csv]

  # Définir le nombre de processus
  nb_processus = 3 

  # Déterminer la taille de chaque lot
  taille_lot = len(noms) // nb_processus

#   print(taille_lot)
  # Définir les débuts et fins de chaque lot
  debuts = [(i * taille_lot)  for i in range(nb_processus)]
  fins = [debut + taille_lot - 1 for debut in debuts]
  fins[-1] = len(noms)

  print(debuts)
  print(fins)

  # Créer et démarrer les processus
  processus = []
  for i in range(nb_processus):
    processus.append(multiprocessing.Process(
      target=traiter_lot,
      args=("etudiants_notes_10000.csv", noms, debuts[i], fins[i]))
    )
    processus[i].start()

  # Attendre la fin de tous les processus
  for processus in processus:
    processus.join()

# Appeler la fonction principale
if __name__ == "__main__":
  main()



[0, 3, 6]
[2, 5, 10]
1 Nsapu Charlie: 55.56 % 

2 Kukwabantu Bob: 41.11 % 

3 Yav Joseph: 47.22 % 
4 Amani Rachel: 39.44 % 


5 Kukwabantu Alice: 48.89 % 

7 Nzeba Mariella: 43.89 % 
6 Buana Joel: 51.67 % 

8 Etumba Hugo: 65.56 % 


9 Mbala Diana: 59.44 % 

10 Manzombi Gina: 32.22 % 



Rendons le code plus dynamique en créant des processus en fonctions du nombre de coeurs sur la machine

In [163]:
import csv
import multiprocessing
from itertools import islice

def nombre_coeurs():
  return multiprocessing.cpu_count()

# Fonction pour calculer le pourcentage d'un étudiant
def calculer_pourcentage(notes):
  total = sum(notes)
  pourcentage = (total / (len(notes) * 20)) * 100
  return round(pourcentage, 2)

# Fonction pour traiter un lot d'étudiants
def traiter_lot(fichier_csv, noms, debut, fin):
  with open(fichier_csv, "r", newline="") as fichier:
    lecteur_csv = csv.reader(fichier)
    # Sauter l'en-tête
    lecteur_csv = islice(lecteur_csv, 1, None)
    # Itérer sur les étudiants du lot
   
    for i, ligne in enumerate(lecteur_csv):
      # print(debut)
      # print(fin)
      # print("\n")
      if i not in range(debut, fin + 1 ): 
        continue
      # Extraire les notes
      notes = [int(note) for note in ligne[1:]]
      # Calculer le pourcentage
      pourcentage = calculer_pourcentage(notes)
      # Afficher le nom de l'étudiant et son pourcentage
      print(f"{noms[i]}: {pourcentage} % \n")

# Fonction principale
def main():
  # Ouvrir le fichier CSV des étudiants en lecture
  with open("etudiants_notes_10000.csv", "r", newline="") as fichier_csv:
    # Lire les noms des étudiants
    lecteur_csv = islice(csv.reader(fichier_csv), 1, None)

    noms = [ligne[0] for ligne in lecteur_csv]

  # Définir le nombre de processus
  nb_processus = nombre_coeurs()

  # Déterminer la taille de chaque lot
  taille_lot = len(noms) // nb_processus

#   print(taille_lot)
  # Définir les débuts et fins de chaque lot
  debuts = [(i * taille_lot)  for i in range(nb_processus)]
  fins = [debut + taille_lot - 1 for debut in debuts]
  fins[-1] = len(noms)

  print(debuts)
  print(fins)

  # Créer et démarrer les processus
  processus = []
  for i in range(nb_processus):
    processus.append(multiprocessing.Process(
      target=traiter_lot,
      args=("etudiants_notes_10000.csv", noms, debuts[i], fins[i]))
    )
    processus[i].start()

  # Attendre la fin de tous les processus
  for processus in processus:
    processus.join()

# Appeler la fonction principale
if __name__ == "__main__":
  main()


[0, 3, 6]
[2, 5, 10]
1 Nsapu Charlie: 55.56 % 

2 Kukwabantu Bob: 41.11 % 

3 Yav Joseph: 47.22 % 
4 Amani Rachel: 39.44 % 


5 Kukwabantu Alice: 48.89 % 

6 Buana Joel: 51.67 % 
7 Nzeba Mariella: 43.89 % 


8 Etumba Hugo: 65.56 % 

9 Mbala Diana: 59.44 % 

10 Manzombi Gina: 32.22 % 

